<center> <img src="https://storage.cloud.google.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="300"/> </center>

# <center>Getting Started with Hosted Phoenix

This guide demonstrates how to use hosted phoenix, our solution for AI developers to trace and evaluate their LLM applications without setting up infrastructure. Read more about Phoenix [here](https://docs.arize.com/phoenix/).

ℹ️ This notebook requires an OpenAI API key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies. It does NOT require installing arize-phoenix to log traces, as we are compliant with OpenTelemetry tracing.

In [ ]:
!pip install -q "arize-phoenix>=4.29.0" openinference-instrumentation-openai openai

In [ ]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
import os
from getpass import getpass

if not (PHOENIX_API_KEY := os.getenv("PHOENIX_API_KEY")):
    PHOENIX_API_KEY = getpass("🔑 Enter your Phoenix API key: ")

os.environ["PHOENIX_API_KEY"] = PHOENIX_API_KEY

## Step 2: Setup Tracing
Let's send a trace to Hosted Phoenix!

In [ ]:
from openinference.instrumentation.openai import OpenAIInstrumentor

from phoenix.otel import register

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

# Setup OTEL tracing for hosted Phoenix. The register function will automatically detect the endpoint and headers from your environment variables.
tracer_provider = register()

# Turn on instrumentation for OpenAI
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)

Send query to Hosted Phoenix


In [ ]:
import openai

openai_client = openai.OpenAI()
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Write a haiku."}],
    max_tokens=20,
)
print(response.choices[0].message.content)

## Step 3: Access Phoenix Instance

From here, you can access your Phoenix instance to power evaluations, experiments, upload datasets, etc. The code below queries spans from the last 7 days, and exports them to a pandas dataframe.


In [ ]:
from datetime import datetime, timedelta

import phoenix as px

# Initiate Phoenix client
px_client = px.Client()

# Get spans from the last 7 days only
start = datetime.now() - timedelta(days=7)

# Get spans to exclude the last 24 hours
end = datetime.now() - timedelta(days=0)

phoenix_df = px_client.query_spans(start_time=start, end_time=end)
phoenix_df.head()